<a href="https://colab.research.google.com/github/stephenfrein/csc8491/blob/main/Homework4_Frein.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Part 1

In [ ]:
fuzzycars = read.csv2('https://csc8491.s3.amazonaws.com/car_data.csv', stringsAsFactors = TRUE)

In [ ]:
set.seed(12345678)
sample = sample(1:nrow(fuzzycars), 0.8 * nrow(fuzzycars))
cars.train = fuzzycars[sample, ]
cars.test = fuzzycars[-sample, ]
sprintf("Training rows: %s", nrow(cars.train))
sprintf("Testing rows: %s", nrow(cars.test))

In [ ]:
install.packages("tree")

In [ ]:
head(cars.train)

In [ ]:
library(tree)
cars.tree = tree(acceptability ~ ., cars.train)
summary(cars.tree)

In [ ]:
plot(cars.tree)
text(cars.tree, pretty=0)


In [ ]:
cars.preds = predict(cars.tree, cars.test, type="class")
cars.actuals = cars.test$acceptability
table(cars.preds, cars.actuals)

In [ ]:

cm = as.data.frame(table(cars.preds, cars.actuals))
cm


In [ ]:
print("Accuracy:")
# number correctly predicted                       / total number
sum(cm[cm$cars.preds==cm$cars.actuals, c('Freq')]) / nrow(cars.test)

In [ ]:
print("Precision:")
# assume "acceptable" is positive class
# number correctly predicted as acceptable                        /  number predicted as acceptable
sum(cm[cm$cars.preds=="acc" & cm$cars.actuals=="acc", c('Freq')]) / sum(cm[cm$cars.preds=="acc",c('Freq')])

In [ ]:
print("Recall:")
# assume "acceptable" is positive class
# number correctly predicted as acceptable                        /  total acceptable
sum(cm[cm$cars.preds=="acc" & cm$cars.actuals=="acc", c('Freq')]) / sum(cm[cm$cars.actuals=="acc",c('Freq')])

Accuracy was higher in summary() function. This isn't surprising, since summary reports accuracy on the data the model was trained on.

In [ ]:
new_car = data.frame(purchase_price=as.factor(c("high")), maint_cost=as.factor(c("high")), num_doors=c(2), num_persons=c(4), trunk_size=as.factor(c("small")), safety_rating=as.factor(("high")))
new_car

In [ ]:
predict(cars.tree, new_car, type="class")
predict(cars.tree, new_car)

In [ ]:
install.packages("randomForest")
library(randomForest)

In [ ]:
cars.rf = randomForest(acceptability~., data=cars.train, importance = TRUE)
cars.rf

In [ ]:
cars.preds.rf = predict(cars.rf, cars.test, type="class")
table(cars.preds.rf, cars.actuals)

In [ ]:
cm = as.data.frame(table(cars.preds.rf, cars.actuals))
cm

In [ ]:
print("Accuracy:")
# number correctly predicted                       / total number
sum(cm[cm$cars.preds.rf==cm$cars.actuals, c('Freq')]) / nrow(cars.test)

In [ ]:
print("Precision:")
# assume "acceptable" is positive class
# number correctly predicted as acceptable                        /  number predicted as acceptable
sum(cm[cm$cars.preds.rf=="acc" & cm$cars.actuals=="acc", c('Freq')]) / sum(cm[cm$cars.preds.rf=="acc",c('Freq')])

In [ ]:
print("Recall:")
# assume "acceptable" is positive class
# number correctly predicted as acceptable                        /  total acceptable
sum(cm[cm$cars.preds.rf=="acc" & cm$cars.actuals=="acc", c('Freq')]) / sum(cm[cm$cars.actuals=="acc",c('Freq')])

In [ ]:
importance(cars.rf)
varImpPlot(cars.rf)


Part 2

In [ ]:
# H = laptop will fail
# E - employee is from sales
#P(H|E)        = P(E|H) * P(H) / P(E)
posterior_prob = .15    * .05  / .08
sprintf("Percent chance the laptop will fail: %s%%", posterior_prob * 100)

Part 3

In [ ]:
twts = read.csv("https://csc8491.s3.amazonaws.com/nova_tweets.csv")
str(twts)

In [ ]:
twts$tod = as.factor(ifelse((twts$tod>=20 | twts$tod < 6),"night","day"))
str(twts)
table(twts$tod)
prop.table(table(twts$tod))

In [ ]:
head(twts)


In [ ]:
# build a corpus (body of docs) using the text mining (tm) package
install.packages("tm")
install.packages("SnowballC")
library(tm)

In [ ]:
# VCorpus is an in-memory (volatile) corpus – simplest choice here
twt_corpus <- VCorpus(VectorSource(twts$tweet_txt))
twt_corpus_clean <- tm_map(twt_corpus, content_transformer(tolower))
# remove numbers
twt_corpus_clean <- tm_map(twt_corpus_clean, removeNumbers)
# remove punctuation
twt_corpus_clean <- tm_map(twt_corpus_clean, removePunctuation)
# remove stop words
twt_corpus_clean <- tm_map(twt_corpus_clean, removeWords, stopwords())
# now stem our actual corpus
twt_corpus_clean <- tm_map(twt_corpus_clean, stemDocument)
# eliminate unneeded whitespace
twt_corpus_clean <- tm_map(twt_corpus_clean, stripWhitespace)
# examine the final clean corpus
lapply(twt_corpus[1:3], as.character)
lapply(twt_corpus_clean[1:3], as.character)


In [ ]:
# create a document-term sparse matrix
twt_dtm <- DocumentTermMatrix(twt_corpus_clean)
# creating training and test datasets
train_pct <- .8
set.seed(123)
train = sample(1:nrow(twt_dtm), train_pct * nrow(twt_dtm))
twt_dtm_train <- twt_dtm[train, ]
twt_dtm_test  <- twt_dtm[-train, ]


In [ ]:
# also save the labels
twt_train_labels <- twts[train, ]$tod
twt_test_labels  <- twts[-train, ]$tod
# check that the proportion of spam is similar
prop.table(table(twt_train_labels))
prop.table(table(twt_test_labels))

In [ ]:
# save frequently-appearing terms to a character vector
twt_freq_words <- findFreqTerms(twt_dtm_train, 5)
str(twt_freq_words)

In [ ]:
# create DTMs with only the frequent terms
twt_dtm_freq_train <- twt_dtm_train[ , twt_freq_words]
twt_dtm_freq_test <- twt_dtm_test[ , twt_freq_words]

In [ ]:
# convert counts to presence indicator - # times not important
convert_counts <- function(x) {
  return(ifelse(x > 0, "Yes", "No"))
}
# apply() convert_counts() to columns of train/test data
twt_train <- apply(twt_dtm_freq_train, MARGIN = 2, convert_counts)
twt_test  <- apply(twt_dtm_freq_test, MARGIN = 2, convert_counts)

In [ ]:
# install naive bayes packages
install.packages("e1071")
library(e1071)

In [ ]:
twt_classifier <- naiveBayes(twt_train, twt_train_labels)
# evaluating model performance ----
twt_test_pred <- predict(twt_classifier, twt_test)
table(twt_test_pred, twt_test_labels)

In [ ]:
cm = as.data.frame(table(twt_test_pred, twt_test_labels))
typeof(cm)
cm

In [ ]:
print("Precision:")
# assume "day" is positive class
# number correctly predicted as day                                     /  number predicted as day
sum(cm[cm$twt_test_pred=="day" & cm$twt_test_labels=="day", c('Freq')]) / sum(cm[cm$twt_test_pred=="day",c('Freq')])

In [ ]:
print("Recall:")
# assume "day" is positive class
# number correctly predicted as day                                     /  total number of days
sum(cm[cm$twt_test_pred=="day" & cm$twt_test_labels=="day", c('Freq')]) / sum(cm[cm$twt_test_labels=="day",c('Freq')])

In [ ]:
nrow(twt_test)